In [1]:
import numpy as np
#from BPmodule.BPGen import BIN_CLSI,BIN_CLSVII,BIN_CLSVIII,BIN_CLSV,BIN_CLSIV,BIN_CLSVI
#from BPmodule.BPGen import CLASS_I50,CLASS_I100,CLASS_I150,CLASS_I1V00,CLASS_I200,CLASS_IV200,CLASS_IV50,CLASS_V100,CLASS_V200,CLASS_V50,CLASS_VI100,CLASS_VI200,CLASS_VI50,CLASS_VII100,CLASS_VII200,CLASS_VII50,CLASS_VIII100,CLASS_VIII200,CLASS_VIII50
from BPmodule.BPGen import CreateData
import pandas as pd
from numba import njit, objmode, prange
from numba.typed import Dict
from numba.core import types
import time
from numba.typed import List as NumbaList
import matplotlib.pyplot as plt
from typing import List
from numpy import savetxt
from BPnumba.GeneticOperators import CreateHeuristicPob,InstancePob,Ind
from BPnumba.Bench import Test
from BPnumba.DFFA import createDFFA
from BPnumba.EDFFA import createEDFFA,EAlphaStep,EAlphaStepC1
from BPnumba.DABC import createDABC
from BPnumba.EDABC import createEDABC 
from BPnumba.AGH import createAG,NAG  

In [2]:
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)

createText = lambda x,y : savetxt(
    fname = "Results/"+x + ".csv",
    X= y,
    delimiter=", ",
    header=" Fi BoxesPacked MaxItr Time(s) | 50 individuos",
    fmt='%g'
)

numTest = 20
num = 4
agL = zerovec(num, 2*numTest)
dabcL = zerovec(num, 2*numTest)
ffaL = zerovec(num, 2*numTest)

pr_selec = 0.85
pr_cross = 0.75
maxItr=1000
pop = 50
algoritms = [1,2]
problems = [3]


In [4]:
contenedor,datos = CreateData(algorithm=1,problem=1)

start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True)
Test(datos.copy(),contenedor,aga, maxItr, agL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 1 ",minutes,":",seconds)
start = time.time()
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1)
Test(datos.copy(),contenedor,aga, maxItr, agL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin ag 2 ",minutes,":",seconds)

start = time.time()
dabc = createDABC()
Test(datos.copy(),contenedor,dabc, maxItr, dabcL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dabc 1 ",minutes,":",seconds)
start = time.time()
dabc = createDABC(1)
Test(datos.copy(),contenedor,dabc, maxItr, dabcL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dabc 2 ",minutes,":",seconds)

start = time.time()
dabc = createEDABC()
Test(datos.copy(),contenedor,dabc, maxItr, dabcL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edabc 1 ",minutes,":",seconds)
start = time.time()
dabc = createEDABC(1)
Test(datos.copy(),contenedor,dabc, maxItr, dabcL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edabc 2 ",minutes,":",seconds)

start = time.time()
effa = createDFFA()
Test(datos.copy(),contenedor,effa, maxItr, ffaL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dffa 1 ",minutes,":",seconds)
start = time.time()
effa = createDFFA(1)
Test(datos.copy(),contenedor,effa, maxItr, ffaL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin dffa 2 ",minutes,":",seconds)

start = time.time()
effa = createEDFFA()
Test(datos.copy(),contenedor,effa, maxItr, ffaL)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 1 ",minutes,":",seconds)
start = time.time()
effa = createEDFFA(1)
Test(datos.copy(),contenedor,effa, maxItr, ffaL,1)
minutes = round((time.time()-start)/ 60)
seconds = int((time.time()-start) % 60)
print("fin edffa 2 ",minutes,":",seconds)


fin ag 1  0 : 0
fin ag 2  0 : 0
fin dabc 1  0 : 0
fin dabc 2  0 : 0
fin edabc 1  0 : 0
fin edabc 2  0 : 0
fin dffa 1  0 : 3
fin dffa 2  0 : 3
fin edffa 1  0 : 0
fin edffa 2  0 : 0


In [ ]:
aga = createAG(ps=pr_selec,pc=pr_cross,adaptive=True)
aga2 = createAG(ps=pr_selec,pc=pr_cross,adaptive=True,heuristic=1)

for problem in problems:
    for algoritm in algoritms:
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor,aga, maxItr, agL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1HAG",agL)
        Test(datos.copy(),contenedor,aga2, maxItr, agL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1HAG",agL)

In [ ]:

effa = createDFFA()
effa2 = createDFFA(heuristic= 1)

for problem in problems:
    
    for algoritm in algoritms:

        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        
        createText("P"+str(problem)+"A"+str(algoritm)+"/DFFA",ffaL)
        
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        
        createText("P"+str(problem)+"A"+str(algoritm)+"/DFFA",ffaL)

In [ ]:
algoritms = [1,2]
problems = [1,2]

effa = createEDFFA()
effa2 = createEDFFA(heuristic= 1)

for problem in problems:
    for algoritm in algoritms:
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, effa,maxItr, ffaL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1EDFFA",ffaL)
        Test(datos.copy(),contenedor, effa2,maxItr, ffaL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/c1EDFFA",ffaL)

In [ ]:
algoritms = [1,2]
problems = [2]

dabc = createEDABC()
dabc2 = createEDABC(1)

for problem in problems:
    for algoritm in algoritms:
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, dabc,maxItr, dabcL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDABC",dabcL)
        Test(datos.copy(),contenedor, dabc2,maxItr, dabcL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/EDABC",dabcL)

In [ ]:
dabc = createDABC()
dabc2 = createDABC(1)

for problem in problems:
    for algoritm in algoritms:
        contenedor,datos = CreateData(algorithm=algoritm,problem=problem)
        Test(datos.copy(),contenedor, dabc,maxItr, dabcL)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DABC",dabcL)
        Test(datos.copy(),contenedor, dabc2,maxItr, dabcL,1)
        createText("P"+str(problem)+"A"+str(algoritm)+"/DABC",dabcL)